# Machine Learning
## Group 4
## Assignment 4
## K-means clustering


## 2. Web scraping

In [8]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import unicodedata

from sklearn.cluster import KMeans
import numpy as np

In [9]:
attributes = ['Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Dribbling', 'Curve', 'Free Kick Accuracy',
              'Long Passing','Ball Control', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance', 'Shot Power','Jumping',
              'Stamina', 'Strength','Long Shots','Aggression','Interceptions','Positioning','Vision','Penalties','Composure','Marking',
              'Standing Tackle','Sliding Tackle','GK Diving','GK Handling', 'GK Kicking','GK Positioning','GK Reflexes']

              
links = []
for offset in ['0','100','200']:
    page = requests.get('http://sofifa.com/players?na=52&offset=' + offset)
    soup = BeautifulSoup(page.content,'html.parser')
    for link in soup.find_all('a'):
        links.append(link.get('href'))

links = ['http://sofifa.com' + l for l in links if 'player/' in l]

pattern = r"""\s*([\w\s]*)"""

for attr in attributes:
    pattern += r""".*?(\d*\s*"""+attr+r""")""" #Attribute variables
    
pat = re.compile(pattern, re.DOTALL)

rows = []
for j,link in enumerate(links):
    print(j, link)
    row = [link]
    playerpage = requests.get(link)
    playersoup = BeautifulSoup(playerpage.content, 'html.parser')
    text = playersoup.get_text()
    #text = str(unicodedata.normalize('NFKD',text).encode('ascii','ignore'))
    a = pat.match(text)
    row.append(a.group(1))
    for i in range(2,len(attributes)+2):
        row.append(int(a.group(i).split()[0]))
    rows.append(row)
    #print(row[1])
df = pd.DataFrame(rows,columns =['link','name'] + attributes)
df.to_csv('ArgentinaPlayers.csv',index=False)

0 http://sofifa.com/player/158023
1 http://sofifa.com/player/167664
2 http://sofifa.com/player/153079
3 http://sofifa.com/player/183898
4 http://sofifa.com/player/211110
5 http://sofifa.com/player/184144
6 http://sofifa.com/player/192366
7 http://sofifa.com/player/178562
8 http://sofifa.com/player/142754
9 http://sofifa.com/player/201399
10 http://sofifa.com/player/191180
11 http://sofifa.com/player/215316
12 http://sofifa.com/player/201862
13 http://sofifa.com/player/186547
14 http://sofifa.com/player/142780
15 http://sofifa.com/player/170481
16 http://sofifa.com/player/142784
17 http://sofifa.com/player/158963
18 http://sofifa.com/player/176915
19 http://sofifa.com/player/193061
20 http://sofifa.com/player/170368
21 http://sofifa.com/player/143076
22 http://sofifa.com/player/183893
23 http://sofifa.com/player/183900
24 http://sofifa.com/player/179847
25 http://sofifa.com/player/165536
26 http://sofifa.com/player/143587
27 http://sofifa.com/player/173306
28 http://sofifa.com/player/19

Explanations here

## 3. 500 English players

We need to modify the code to parse players from a different country. We also need to modify the number of players parsed. This is done by modifying the first for loop into this form. We change the list of offset from ['0','100','200'] into ['0','100','200','300','400']. This adds 2 more pages of 100 players for the code to search through.

Next, we modify the base website to be scraped, from na=52 into na=14, where na=14 corresponds with the England players. The modified loop is shown below

In [10]:
#for offset in ['0','100','200','300','400']:
#    page = requests.get('http://sofifa.com/players?na=14&offset=' + offset)
#    soup = BeautifulSoup(page.content,'html.parser')
#    for link in soup.find_all('a'):
#        links.append(link.get('href'))

## 4. Training the model

In [11]:
#Drop link and name from variables to train on
dfX = df.drop(['link','name'],axis=1)

#Train the model
football = KMeans(n_clusters=5,random_state=12345).fit(dfX)

## 5. Labelling the clusters

In [24]:
labels = football.labels_

#Boolean list, indicates belonging to each cluster
g1 = labels==0
g2 = labels==1
g3 = labels==2
g4 = labels==3
g5 = labels==4

#Extracting players into their respective clusters
c1 = df.ix[g1,]
c2 = df.ix[g2,]
c3 = df.ix[g3,]
c4 = df.ix[g4,]
c5 = df.ix[g5,]

### Cluster 1

In [25]:
print(c1['name'])

6           Nicolás Otamendi 
14         Gonzalo Rodríguez 
15            Ezequiel Garay 
24            Federico Fazio 
25            Gustavo Cabral 
29        Federico Fernández 
30            Lisandro López 
41             Víctor Cuesta 
46         Martín Demichelis 
52       Santiago Gentiletti 
63          Nicolás Burdisso 
68           Germán Pezzella 
72          Mauro Dos Santos 
88           Jonatan Maidana 
93          Santiago Vergini 
106           Matías Caruzzo 
109            Luciano Lollo 
111         Martín Mantovani 
113    Julio Alberto Barroso 
120         Jonathan Schunke 
125           Nicolás Spolli 
135          Marcos Angeleri 
137           Renato Civelli 
143         Juan Insaurralde 
146        Carlos Izquierdoz 
150          Matías Zaldivia 
157       José María Basanta 
164         Leandro Desábato 
173              Juan Forlin 
176           Fernando Tobio 
180       Guillermo Burdisso 
185              Daniel Díaz 
193             Germán Conti 
196       

### Cluster 2

In [27]:
print(c2['name'])

7                Éver Banega 
8          Javier Mascherano 
12               Marcos Rojo 
13           Mateo Musacchio 
16            Pablo Zabaleta 
17              Lucas Biglia 
18         Augusto Fernández 
19           Roberto Pereyra 
22             Claudio Yacob 
27          Cristian Ansaldi 
36            Nicolás Pareja 
47             Guido Pizarro 
48              Marcos Acuña 
49                Enzo Pérez 
57             David Abraham 
65               Pablo Pérez 
67              Lucas Castro 
69           Leandro Paredes 
75         Esteban Cambiasso 
76              Gino Peruzzi 
77             Fernando Gago 
81         Facundo Roncaglia 
82              Emmanuel Mas 
84        Matías Kranevitter 
87               Óscar Trejo 
89         Ramiro Funes Mori 
90         Gonzalo Escalante 
96           Leonardo Ponzio 
97        Nicolás Tagliafico 
99           Leonel Vangioni 
                ...          
214             Juan Mercier 
216              Jorge Ortiz 
221       

### Cluster 3

In [29]:
print(c3['name'])

11           Gerónimo Rulli 
32            Sergio Romero 
37         Marcelo Barovero 
43            Nahuel Guzmán 
44          Willy Caballero 
53        Sebastián Torrico 
54        Agustín Marchesín 
64          Mariano Andújar 
66            Franco Armani 
83            Agustín Orión 
105         Mariano Barbosa 
108        Fernando Monetti 
123              Germán Lux 
126         Albano Bizzarri 
147      Juan Pablo Carrizo 
162    Cristian Campestrini 
174          Guillermo Sara 
183        Luciano Pocrnjic 
192             Rodrigo Rey 
203             Marcos Díaz 
215           Javier García 
229             Jorge Broun 
235            Oscar Ustari 
243          Julián Speroni 
253            Luis Ardente 
290         Nereo Fernández 
Name: name, dtype: object


### Cluster 4

In [31]:
print(c4['name'])

1          Gonzalo Higuaín 
9             Mauro Icardi 
28          Eduardo Salvio 
34           Pablo Batalla 
45            Lucas Alario 
55             Marco Ruben 
59          Nicolás Blandi 
61         Darío Benedetto 
62             Gustavo Bou 
78           Mauro Boselli 
95         Franco Di Santo 
100       Jonathan Calleri 
116          Silvio Romero 
118          Emiliano Sala 
124      Maximiliano López 
138       Facundo Ferreyra 
141      Sebastián Driussi 
151            Julio Furch 
155           Lucas Viatri 
156         Guido Carrillo 
163              José Sand 
165           Germán Denis 
167       Enrique Triverio 
170     Juan Ignacio Gomez 
171         Leonardo Ulloa 
179            Franco Jara 
184       Giovanni Simeone 
187         Sebastián Leto 
190     Juan Martín Lucero 
191            Mauro Matos 
211        Milton Caraglio 
212           Diego Churín 
219    Denis Stracqualursi 
236         Mariano Pavone 
239         Ezequiel Ponce 
265        Darío Cvi

### Cluster 5

In [34]:
print(c5['name'])

0                 Lionel Messi 
2                Sergio Agüero 
3               Ángel Di María 
4                 Paulo Dybala 
5               Nicolás Gaitán 
10              Javier Pastore 
20                 Erik Lamela 
21             Alejandro Gómez 
23               Diego Perotti 
26          Fernando Belluschi 
31              Manuel Lanzini 
33              Luciano Vietto 
35              Lisandro López 
38             Rodrigo Palacio 
39                Ángel Correa 
40              Ignacio Piatti 
42                   José Sosa 
50                Diego Valeri 
51                Pablo Piatti 
56            Diego Buonanotte 
58          Rogelio Funes Mori 
60             Pablo De Blasis 
70                Franco Cervi 
71                Mauro Zárate 
73              Lautaro Acosta 
74            Sebastián Blanco 
79                 Ismael Sosa 
80       Alexander Szymanowski 
85              Matías Delgado 
86                  Mauro Díaz 
                 ...           
194     

## 6. Prediction of new point

In [35]:
#Retrieve centroids
centroid = football.cluster_centers_

#Parameters of new points, in this order, 'Crossing','Finishing','Sprint Speed','Long Shots','Aggression','Marking','GK Handling'
newPoint = [45,40,40,35,45,60,15]

#Set names of columns we want
var = set(['Crossing','Finishing','Sprint Speed','Long Shots','Aggression','Marking','GK Handling'])

#Find index of columns we want
varLocations = [i for i,x in enumerate(list(df)) if x in var]

#Only use columns with index matching above in that order
centroidSub = [x[varLocations] for x in centroid]

#Calculate distances of point from each centroid
distances = [np.linalg.norm(x-newPoint) for x in centroidSub]

#Find index of centroid with smallest distance, then +1 to find cluster number
distances.index(min(distances))+1

5

We group the new point into Cluster 5.